In [31]:
import os
from PIL import Image
import tensorflow.keras
import cv2
import numpy as np
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt

In [8]:
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from sklearn.metrics.pairwise import cosine_similarity as cs
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [3]:
#pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\nikol\.venvs\lewagon\scripts\python.exe -m pip install --upgrade pip' command.


In [19]:
def file_selector(folder_path):
    filenames = []
    for image in os.listdir(folder_path):
        if image[-3:] == 'jpg' or image[-3:] == 'png' or image[-4:] == 'jpeg':
            filenames.append(os.path.abspath(folder_path + "/" + image))
    return filenames


In [11]:
model = MobileNetV2(weights = "imagenet", include_top = True, input_shape = (224, 224, 3))

14540800/14536120 [==============================] - 1s 0us/step


In [12]:
def image_vectorization(image_path):
  #print(image_path)
    img=Image.open(image_path)
    img=img.convert("RGB")
    img = img.resize((224, 224))
    #print(img.size)
    img = img_to_array(img)
    img = img.reshape((-1, 224, 224, 3))
    #print(img.shape)
    img=preprocess_input(img)
    vector=model.predict(img)

    return vector

In [14]:
def get_vectors(files):   
    liste_vector=[]
    for picture in files:
        liste_vector.append(image_vectorization(picture)[0].tolist())
    return liste_vector

In [27]:
def similarity_detection(vectors, files):
  #wenn iteration in skip, dann alles überspringen
  skip=[]
  dict_pairs={}
  X=np.array(vectors)#similarity bestimmen
  similarity=cs(X)#similarity bestimmen
  #for schleife die über alle listen (Bilder) in similarity geht
  for row in similarity.tolist():
    if similarity.tolist().index(row) in skip:
      pass
    else:
  #wir suchen nach dem index der Bilder die ähnlich zu dem jetzigen Bild (der Iteration) ist
      index_list=[index for index in range(len(row)) if row[index] >= 0.3]
      #print(index_list)
  #skip.append index
      liste_pairs=[]
      for index in index_list:
        skip.append(index)
        liste_pairs.append(files[index])
      #print(skip)
  #dictionary mit ähnlichen Bildern wie in alter Funktion
      dict_pairs[f'{similarity.tolist().index(row)}']=liste_pairs
  
  return dict_pairs

In [35]:
def liste_similar_pictures(dictionary):
  liste=[]
  for (key, value) in dictionary.items():
    if len(value)>1:
      for picture in value[1:]:
        liste.append(picture)
  return liste

In [ ]:
def replace_pictures(liste):
    for img in liste:
        os.replace(f"/content/gdrs.repive/MyDrive/Apollo_20/Test-similarity/{picture}", f"/content/gdrive/MyDrive/Apollo_20/delete_similar_pictures/{picture}")
  
    return True

In [23]:
files_nick=file_selector("/Users/Nikol/code/NikolausKCL/Test-similarity")

In [24]:
vector_liste=get_vectors(files_nick)

In [28]:
similarity_dict=similarity_detection(vector_liste, files_nick)

In [36]:
liste_sim=liste_similar_pictures(similarity_dict)

In [37]:
liste_sim

['C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\7d119d43-b6a7-44ca-902e-fe89a4f0a988.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\edcbf496-e53d-4788-aace-bbe2da369fff.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\6c0970ca-b0d9-4192-a9a3-14d03764da94.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\8560462d-963e-4d8c-b9a0-05bd5a12cf8e.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\dfb1fbbc-edef-4fa1-b1ef-0b035c8e53e9.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\edcbf496-e53d-4788-aace-bbe2da369fff.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\edcbf496-e53d-4788-aace-bbe2da369fff.jpg',
 'C:\\Users\\Nikol\\code\\NikolausKCL\\Test-similarity\\c0725d84-72e1-4b0f-8d0c-14fc37614250.jpg']

In [38]:
def execute(folder_path):
    files=file_selector(folder_path)
    vector_liste=get_vectors(files)
    similarity_dict=similarity_detection(vector_liste, files)
    liste_sim=liste_similar_pictures(similarity_dict)
    os.mkdir(f"{folder_path}/similar_pictures_folder")
    similar_pictures_folder = os.path.abspath(f"{folder_path}/similar_pictures_folder")
    for picture in liste_sim:
        os.replace(f"{picture}", f"{similar_pictures_folder}/{picture.rsplit('/', 1)[-1]}")